In [1]:
# credits to Dinesh Daultani (@dineshdaultani)

import requests
import json

#api = NewsApiClient(api_key='0763606ae56d49e08902365e0dbcb239')
#sources = api.sources(params)
#articles = api.articles(sources[0]['id'], params) 

In [2]:
class ArchiveAPI(object):
    def __init__(self, key=None):
        self.key = key
        """
        Initializes the ArchiveAPI class. Raises an exception if no API key is given.
        :param key: New York Times API Key
        """
        self.root = 'http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}' 
        if not self.key:
            nyt_dev_page = 'http://developer.nytimes.com/docs/reference/keys'
            exception_str = 'API Key required.{}'
            raise NoAPIKeyException(exception_str.format(nyt_dev_page))

    def query(self, year=None, month=None, key=None,):
        """
        Calls the archive API and returns the results as a dictionary.
        :param key: Defaults to the API key used to initialize the ArchiveAPI class.
        """
        if not key: key = self.key
        if (year < 1882) or not (0 < month < 13):
            # currently the Archive API only supports year >= 1882
            exception_str = 'Invalid query: See http://developer.nytimes.com/archive_api.json'
            raise InvalidQueryException(exception_str)
        url = self.root.format(year, month, key)
        r = requests.get(url)
        return r.json()

In [3]:
my_api = ArchiveAPI('0ba6dc04a8cb44e0a890c00df88c393a')

In [6]:
years = [2017, 2016] # 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007]
months = [1, 2] #, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for year in years:
    for month in months:
        mydict = my_api.query(year, month)
        file_str = '/home/ostapkharysh/Documents/bt_data/NYT_news/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str, 'w') as fout:
            json.dump(mydict, fout)
        fout.close()

In [7]:
import pandas as pd
with open('/home/ostapkharysh/Documents/bt_data/NYT_news/2016-02.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)
df

,copyright,response
docs,Copyright (c) 2013 The New York Times Company....,"[{'abstract': None, 'lead_paragraph': 'The Car..."
meta,Copyright (c) 2013 The New York Times Company....,{'hits': 7297}


In [8]:
#  'pub_date', 'web_url','word_count'


df['response']['docs'][5]['word_count']

'255'

In [14]:
k = df['response']['docs'][5]['web_url']
print(k)
print(df['response']['docs'][5]['pub_date'])
df['response']['docs'][5]['word_count']

https://takingnote.blogs.nytimes.com/2016/02/01/so-this-is-the-humble-trump/
2016-02-01T23:42:57Z


'255'

In [15]:
from newspaper import Article
import newspaper

In [36]:
article = Article(k)
try:
    article.download()
except:
    print('***FAILED TO DOWNLOAD***', article.url)
article.parse()

In [37]:
article.publish_date

datetime.datetime(2016, 2, 1, 0, 0)

In [38]:
article.text

'Photo\n\nDonald Trump, to borrow his own word, is unbelievable.\n\nAt around 2:30 on Monday afternoon, at a rally in Cedar Rapids, Iowa, Donald Trump was loudly, confidently and repeatedly proclaiming his impending victory in Iowa. Just before 10 p.m., he was acting like he had never expected to win, and like coming in second was a YUGE accomplishment.\n\n“I absolutely love the people of Iowa,” Mr. Trump said at his campaign headquarters. “Unbelievable.”\n\nMr. Trump said that when he declared his candidacy in June, “I was told by everybody, do not go to Iowa, you could never finish even in the top 10.”\n\n“I said I have to do it,” he said. “And we finished second and I want to tell you something, I’m just honored.”\n\nMr. Trump congratulated the victor, Ted Cruz, for winning and Mike Huckabee for quitting and never mentioned Marco Rubio, who was coming in a very close third on Monday night and should be a focus of Mr. Trump’s attention.\n\nIt’s impossible to know whether Mr. Trump un

In [29]:
article.title

'So This Is the Humble Trump?'